# Global TB Trends Analysis
## Exploratory Data Analysis & Visualization

This notebook analyzes global tuberculosis trends using public health data from Our World in Data (OWID) and the World Health Organization (WHO). It demonstrates professional Python data science workflows including data loading, cleaning, analysis, and visualization of TB incidence, mortality, and program performance indicators.

**Analysis Strategy:**
1. **OWID Data**: High-level trends and geographic analysis of TB incidence and mortality
2. **WHO Data**: Deeper insights into program performance (treatment success rates, case detection)
3. **Combined Analysis**: Purposeful merging to explore TB incidence vs treatment success relationships

## Section 1: Import Required Libraries and Configure Environment

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
import sys
import warnings

warnings.filterwarnings('ignore')

# Configure visualization
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

# Add src to path
src_path = Path().resolve().parent / 'src'
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print("✓ Libraries imported successfully")

## Section 2: Load and Explore Our World in Data (OWID) TB Dataset

In [ ]:
from config import OWID_DATA_PATH, WHO_DATA_PATH
from data_loader import load_owid_data, load_who_data, validate_dataframe

# Load OWID data
owid_raw = load_owid_data(str(OWID_DATA_PATH))
print(f"\n{'='*60}")
print("OWID Dataset Overview")
print(f"{'='*60}")
print(f"Shape: {owid_raw.shape}")
print(f"Columns: {owid_raw.columns.tolist()}")
print(f"\nData Types:\n{owid_raw.dtypes}")
print(f"\nMissing Values (%):\n{(owid_raw.isna().sum() / len(owid_raw) * 100).round(2)}")
print(f"\nFirst rows:\n{owid_raw.head()}")

## Section 3: Clean and Tidy OWID TB Data

In [ ]:
from data_cleaning import clean_owid_data

# Clean OWID data
owid_clean = clean_owid_data(owid_raw)

print(f"\n{'='*60}")
print("OWID Data - After Cleaning")
print(f"{'='*60}")
print(f"Shape: {owid_clean.shape}")
print(f"Rows removed: {len(owid_raw) - len(owid_clean)}")
print(f"Year range: {int(owid_clean['year'].min())} - {int(owid_clean['year'].max())}")
print(f"Countries: {owid_clean['country'].nunique()}")
print(f"\nMissing Values (%):\n{(owid_clean.isna().sum() / len(owid_clean) * 100).round(2)}")
print(f"\nSample cleaned data:\n{owid_clean.head(10)}")

## Section 4: Load and Explore WHO TB Dataset

In [ ]:
# Load WHO data
who_raw = load_who_data(str(WHO_DATA_PATH))
print(f"\n{'='*60}")
print("WHO Dataset Overview")
print(f"{'='*60}")
print(f"Shape: {who_raw.shape}")
print(f"Columns: {who_raw.columns.tolist()}")
print(f"\nData Types:\n{who_raw.dtypes}")
print(f"\nMissing Values (%):\n{(who_raw.isna().sum() / len(who_raw) * 100).round(2)}")
print(f"\nFirst rows:\n{who_raw.head()}")

## Section 5: Clean and Tidy WHO TB Data

In [ ]:
from data_cleaning import clean_who_data

# Clean WHO data
who_clean = clean_who_data(who_raw)

print(f"\n{'='*60}")
print("WHO Data - After Cleaning")
print(f"{'='*60}")
print(f"Shape: {who_clean.shape}")
print(f"Rows removed: {len(who_raw) - len(who_clean)}")
print(f"Year range: {int(who_clean['year'].min())} - {int(who_clean['year'].max())}")
print(f"Countries: {who_clean['country'].nunique()}")
print(f"\nMissing Values (%):\n{(who_clean.isna().sum() / len(who_clean) * 100).round(2)}")
print(f"\nSample cleaned data:\n{who_clean.head(10)}")

## Section 6: Analyze TB Incidence and Mortality Trends (OWID)

In [ ]:
from analysis import owid_global_tb_trends, owid_top_affected_countries

# Analyze global TB trends
print(f"\n{'='*60}")
print("OWID Analysis: Global TB Trends")
print(f"{'='*60}")

trends = owid_global_tb_trends(owid_clean)
if not trends.empty:
    print(f"\nGlobal TB Trends Summary (by year):")
    print(trends.head(10))
    print(f"\n... ({len(trends)} total years)")

# Identify top affected countries
latest_year = int(owid_clean['year'].max())
top_countries = owid_top_affected_countries(owid_clean, year=latest_year, n=10)

print(f"\n{'='*60}")
print(f"Top 10 Most Affected Countries ({latest_year})")
print(f"{'='*60}")
print(top_countries)

## Section 7: Visualize TB Geographic and Temporal Patterns (OWID)

In [ ]:
from visualization import plot_global_tb_trends, plot_top_countries_comparison

# Visualize global trends
print("Creating visualizations...")

if not trends.empty and 'tuberculosis_deaths' in trends.columns:
    plot_global_tb_trends(trends, 'tuberculosis_deaths')
    print("✓ Global TB trends plot saved")

# Visualize top countries
if not top_countries.empty:
    metric_col = top_countries.columns[-1]
    plot_top_countries_comparison(top_countries, latest_year, metric_col)
    print(f"✓ Top countries comparison plot saved")

print("\nVisualizations have been saved to outputs/figures/")

## Section 8: Analyze TB Program Performance Indicators (WHO)

In [ ]:
from analysis import who_treatment_success_trends, who_top_performers, who_program_coverage_analysis

# Analyze treatment success trends
print(f"\n{'='*60}")
print("WHO Analysis: TB Program Performance")
print(f"{'='*60}")

success_trends = who_treatment_success_trends(who_clean)
if not success_trends.empty:
    print(f"\nTreatment Success Rate Trends:")
    print(success_trends.head(10))
    print(f"... ({len(success_trends)} total years)")

# Identify top performing countries
latest_year_who = int(who_clean['year'].max())
performers = who_top_performers(who_clean, year=latest_year_who, n=10)

print(f"\n{'='*60}")
print(f"Top 10 Treatment Performers ({latest_year_who})")
print(f"{'='*60}")
if not performers.empty:
    print(performers)
else:
    print("No treatment success rate data available")

# Program coverage analysis
coverage = who_program_coverage_analysis(who_clean)
if not coverage.empty:
    print(f"\n{'='*60}")
    print(f"TB Program Coverage Indicators")
    print(f"{'='*60}")
    print(coverage.head(10))

## Section 9: Create WHO Program Performance Visualizations

In [ ]:
from visualization import plot_treatment_success_trends, plot_treatment_performers

# Visualize WHO data
if not success_trends.empty:
    plot_treatment_success_trends(success_trends)
    print("✓ Treatment success trends plot saved")

if not performers.empty:
    plot_treatment_performers(performers, latest_year_who)
    print(f"✓ Treatment performers plot saved ({latest_year_who})")

print("\nWHO visualizations saved to outputs/figures/")

## Section 10: Merge and Analyze Combined Metrics (Incidence vs Treatment Success)

In [ ]:
from analysis import compare_incidence_vs_treatment_success

# Perform targeted merge
print(f"\n{'='*60}")
print("Combined Analysis: TB Incidence vs Treatment Success")
print(f"{'='*60}")

merged = compare_incidence_vs_treatment_success(owid_clean, who_clean)

if not merged.empty:
    print(f"\nMerged Dataset: {len(merged)} country-year combinations")
    print(f"Unique countries: {merged['country'].nunique()}")
    print(f"Year range: {int(merged['year'].min())} - {int(merged['year'].max())}")
    print(f"\nSample merged data:")
    print(merged.head(10))
    
    # Basic statistics
    print(f"\n{'='*60}")
    print("Summary Statistics (Merged Data)")
    print(f"{'='*60}")
    print(merged.describe())
else:
    print("No overlapping data available for combined analysis.")

## Section 11: Create Comparative Insight Visualizations

In [ ]:
from visualization import plot_incidence_vs_treatment_scatter

if not merged.empty:
    plot_incidence_vs_treatment_scatter(merged)
    print("✓ Incidence vs treatment success scatter plot saved")
    
    # Correlation analysis
    if 'tuberculosis_incidence_rate' in merged.columns and 'tb_treatment_success_rate' in merged.columns:
        correlation = merged['tuberculosis_incidence_rate'].corr(merged['tb_treatment_success_rate'])
        print(f"\nCorrelation between TB Incidence and Treatment Success: {correlation:.3f}")
        
        if correlation < 0:
            print("→ Countries with higher TB incidence tend to have lower treatment success rates")
        else:
            print("→ Countries with higher TB incidence tend to have higher treatment success rates")

print("\nComparative analysis complete. All visualizations saved to outputs/figures/")

## Section 12: Summary and Key Insights

In [ ]:
print(f"\n{'='*60}")
print("ANALYSIS SUMMARY")
print(f"{'='*60}")

print(f"\n📊 OWID Data:")
print(f"  - Records: {len(owid_clean):,}")
print(f"  - Countries: {owid_clean['country'].nunique()}")
print(f"  - Year range: {int(owid_clean['year'].min())} - {int(owid_clean['year'].max())}")
print(f"  - Focus: TB incidence and mortality trends")

print(f"\n🏥 WHO Data:")
print(f"  - Records: {len(who_clean):,}")
print(f"  - Countries: {who_clean['country'].nunique()}")
print(f"  - Year range: {int(who_clean['year'].min())} - {int(who_clean['year'].max())}")
print(f"  - Focus: TB program performance indicators")

if not merged.empty:
    print(f"\n🔗 Combined Analysis:")
    print(f"  - Merged records: {len(merged):,}")
    print(f"  - Countries with both datasets: {merged['country'].nunique()}")
    print(f"  - Relationship analyzed: TB incidence vs treatment success")

print(f"\n📈 Visualizations Generated:")
print(f"  - Global TB trends over time")
print(f"  - Top affected countries comparison")
print(f"  - Treatment success rate trends")
print(f"  - Country-level performance comparisons")
print(f"  - Incidence vs treatment success scatter plot")
print(f"\n✅ All visualizations saved to outputs/figures/")
print(f"✅ Cleaned datasets saved to data/processed/")
print(f"✅ Analysis pipeline and modules ready for production use")